典型的基础类问题：执行一个查询但是返回非预期的结果。

这类问题的表现形式可能是很明显的错误，也可能是在你明知有匹配记录的情况下却没有返回结果，或者其他应用程序中罕见的行为。

简而言之，本章内容是建立在你充分了解应用程序的运行状况以及查询应该返回结果，或者其他应用程序中罕见的行为。

# 1. 语法错误
- 检查查询的语法错误作为排错的第一步
- 始终尝试查看MySQL服务器最终接受到的查询，不仅调试应用程序代码，要获取查询语句

## 1.1 简单语法错误

语法错误明显，错误消息也很清楚。

## 1.2 困难的语法错误：版本迁移导致的问题：保留字

这是一个版本迁移导致的问题；

该语句在5.0版本中运行正常，但是在5.1 版本中出现错误。

问题的原因在于，在5.1版本中，accessible是一个**保留字**。

该该语句加上引号（反引号还是双引号取决于你的SQL格式），即可重新正常运行：

实际场景中的查询语句看起来可能十分繁琐，包括大段的JOIN和复杂的WHERE条件。

所以，即使是简单的错误在其他大量语句的干扰下也很难被查找出来。

我们当时第一步的任务就是简单复杂的查询，使它变成像刚才看到的那样只有一行SELECT 的语句。

这就是一个最简化测试的示例。

当我们看到简化后只有一行的语句也有同样的Bug时，我们就能很块意识到原有程序是因为保留字的问题而产生了错误。

## 1.3 启用通用查询日志
变量：general_log, log_output

通用查询日志包含MySQL服务器接收到的每条独立查询。

很多产品不会在日常运行中使用该日志，因为它在高负载的情况下增长十分迅速，并且写入日志会消耗MySQL服务器的资源，这些资源很可能用于更重要的用途。

从5.1版本开始，可以临时打开通用查询日志，方便随时记录你需要的查询。

也可以将日志记录在表中，可以帮你方便地分类索引日志文件项，因为你可以像查询其他表一样访问查询日志表：

mysql> show variables like 'log_output';
+---------------+-------+
| Variable_name | Value |
+---------------+-------+
| log_output    | FILE  |
+---------------+-------+
1 row in set (0.00 sec)

mysql> set global log_output='table';
Query OK, 0 rows affected (0.00 sec)

mysql> show variables like 'log_output';
+---------------+-------+
| Variable_name | Value |
+---------------+-------+
| log_output    | TABLE |
+---------------+-------+
1 row in set (0.01 sec)

### 查看通用日志查询表，以查找有问题的查询：